In [ ]:
!pip install transformers seqeval[gpu]
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from sklearn.metrics import accuracy_score
import torch

In [ ]:
data_train = pd.read_csv('/content/drive/MyDrive/NER_data/VLSP-2018-NER/train.csv', encoding='utf-8')
data_test = pd.read_csv('/content/drive/MyDrive/NER_data/VLSP-2018-NER/test.csv', encoding = 'utf-8')

In [ ]:
data_train = data_train[['segmented_tokens', 'segmented_labels']].drop_duplicates().reset_index(drop=True)
data_test = data_test[['segmented_tokens', 'segmented_labels']].drop_duplicates().reset_index(drop=True)

In [ ]:
a = len(data_train['segmented_tokens'])
for i in range(a):
  data_train['segmented_tokens'][i]=eval(data_train['segmented_tokens'][i])
  data_train['segmented_labels'][i]=eval(data_train['segmented_labels'][i])
  

In [ ]:
b = len(data_test['segmented_tokens'])
for i in range(b):
  data_test['segmented_tokens'][i]=eval(data_test['segmented_tokens'][i])
  data_test['segmented_labels'][i]=eval(data_test['segmented_labels'][i])

In [ ]:
train_dataset = Dataset.from_pandas(data_train)
test_dataset = Dataset.from_pandas(data_test)

In [ ]:
labels_list =[]
for i, sublist in enumerate(train_dataset['segmented_labels']):
  for x in sublist:
    if x not in labels_list:
      labels_list.append(x)

In [ ]:
label_list = np.unique(labels_list)
label_list

array(['B-LOCATION', 'B-MISCELLANEOUS', 'B-ORGANIZATION', 'B-PERSON',
       'I-LOCATION', 'I-MISCELLANEOUS', 'I-ORGANIZATION', 'I-PERSON', 'O'],
      dtype='<U15')

In [ ]:
a = len(label_list)
label_encoding_dict = {label_list[a-1-i] : i for i in range(0, len(label_list) ) }
inverse_label_encoding_dict = {v: k for k, v in label_encoding_dict.items()}

In [ ]:
inverse_label_encoding_dict

{0: 'O',
 1: 'I-PERSON',
 2: 'I-ORGANIZATION',
 3: 'I-MISCELLANEOUS',
 4: 'I-LOCATION',
 5: 'B-PERSON',
 6: 'B-ORGANIZATION',
 7: 'B-MISCELLANEOUS',
 8: 'B-LOCATION'}

In [ ]:
model_checkpoint = "vinai/phobert-base"
batch_size = 16
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
#Get the values for input_ids, token_type_ids, attention_mask
MAX_SEQ_LENGTH = 256

def tokenize_and_align_labels(examples):
    label_all_tokens = True
    batch_tokens = examples["segmented_tokens"]
    batch_labels = examples["segmented_labels"]
    assert len(batch_tokens) == len(batch_labels)
    labels = []

    batch_wordpiece_labels = []
    batch_wordpiece_tokens = []
    batch_wordpiece_input_ids = []
    batch_wordpiece_label_ids = []
    batch_attention_mask = []

    for idx in range(len(batch_tokens)):
        tokens = batch_tokens[idx]
        labels = batch_labels[idx]
        wordpiece_labels = []
        wordpiece_tokens = []

        for i, token in enumerate(tokens):
            wordpieces = tokenizer.tokenize(token)
            wordpiece_tokens.extend(wordpieces)
            label = labels[i]
            if label.startswith("B-"):
                wordpiece_labels.extend(
                    [label] + ["I-" + label[2:]] * (len(wordpieces) - 1)
                )
            else:
                wordpiece_labels.extend(
                    [label] * len(wordpieces)
                )
        
        if len(wordpiece_tokens) > MAX_SEQ_LENGTH - 2:
            wordpiece_tokens = wordpiece_tokens[:MAX_SEQ_LENGTH - 2]
            wordpiece_labels = wordpiece_labels[:MAX_SEQ_LENGTH - 2]
        
        # batch_wordpiece_labels.append(wordpiece_labels)
        batch_wordpiece_label_ids.append([label_encoding_dict['O']] + [label_encoding_dict[lb] for lb in wordpiece_labels] + [label_encoding_dict['O']])
        batch_wordpiece_input_ids.append([tokenizer.cls_token_id] + tokenizer.convert_tokens_to_ids(wordpiece_tokens) + [tokenizer.sep_token_id])
        batch_attention_mask.append([1] * (len(wordpiece_tokens) + 2))
    
    return {
        "input_ids": batch_wordpiece_input_ids,
        "labels": batch_wordpiece_label_ids,
        "attention_mask": batch_attention_mask
    }
 

In [ ]:
train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True, batch_size=16)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True, batch_size=16)

Parameter 'function'=<function tokenize_and_align_labels at 0x7f2060e08170> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/583 [00:00<?, ?ba/s]

  0%|          | 0/264 [00:00<?, ?ba/s]

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this mo

In [ ]:

args = TrainingArguments(
    output_dir="/content/drive/MyDrive/NER_task/VLSP-2018/NER_output",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    logging_steps=1500,
    weight_decay=0.01,
    save_total_limit = 1,
    load_best_model_at_end = True
)


In [ ]:
from seqeval.metrics import classification_report

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[inverse_label_encoding_dict[int(p)] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[inverse_label_encoding_dict[int(l)] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    
    report  = classification_report(true_labels,true_predictions)
    
    with open('report_VLSP.txt', 'w') as f:
      f.write(report)

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return results
    
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()
trainer.save_model('/content/drive/MyDrive/NER_task/VLSP-2018/Model/BERT-NER.model')


The following columns in the training set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: segmented_tokens, segmented_labels. If segmented_tokens, segmented_labels are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9315
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5830


Epoch,Training Loss,Validation Loss,Location,Miscellaneous,Organization,Person,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,No log,0.058372,"{'precision': 0.8329604772557793, 'recall': 0.9073923639317628, 'f1': 0.8685847589424572, 'number': 1231}","{'precision': 0.32388663967611336, 'recall': 0.47904191616766467, 'f1': 0.3864734299516908, 'number': 167}","{'precision': 0.7575757575757576, 'recall': 0.8883248730964467, 'f1': 0.8177570093457944, 'number': 1379}","{'precision': 0.9495926680244399, 'recall': 0.9353059177532598, 'f1': 0.9423951490651844, 'number': 1994}",0.829367,0.898554,0.862575,0.984485
2,No log,0.055374,"{'precision': 0.876984126984127, 'recall': 0.8976441917140536, 'f1': 0.8871938980329186, 'number': 1231}","{'precision': 0.34551495016611294, 'recall': 0.6227544910179641, 'f1': 0.4444444444444444, 'number': 167}","{'precision': 0.8268575323790047, 'recall': 0.8796229151559101, 'f1': 0.8524244553759662, 'number': 1379}","{'precision': 0.9341404358353511, 'recall': 0.9674022066198595, 'f1': 0.950480413895048, 'number': 1994}",0.854310,0.911968,0.882198,0.986599
3,0.061600,0.058502,"{'precision': 0.8804780876494024, 'recall': 0.8976441917140536, 'f1': 0.8889782783588093, 'number': 1231}","{'precision': 0.3220858895705521, 'recall': 0.6287425149700598, 'f1': 0.4259634888438134, 'number': 167}","{'precision': 0.841374269005848, 'recall': 0.8346627991298042, 'f1': 0.8380050964688751, 'number': 1379}","{'precision': 0.9420716740304369, 'recall': 0.9623871614844534, 'f1': 0.9521210617712728, 'number': 1994}",0.858404,0.897087,0.877319,0.986607
4,0.061600,0.061587,"{'precision': 0.891497975708502, 'recall': 0.8943948009748173, 'f1': 0.8929440389294404, 'number': 1231}","{'precision': 0.3557692307692308, 'recall': 0.6646706586826348, 'f1': 0.4634655532359081, 'number': 167}","{'precision': 0.7949206349206349, 'recall': 0.9079042784626541, 'f1': 0.8476641841570751, 'number': 1379}","{'precision': 0.9458375125376128, 'recall': 0.9458375125376128, 'f1': 0.9458375125376128, 'number': 1994}",0.850274,0.911759,0.879943,0.986260
5,0.061600,0.065802,"{'precision': 0.874414976599064, 'recall': 0.9106417546709992, 'f1': 0.8921607640270592, 'number': 1231}","{'precision': 0.4031620553359684, 'recall': 0.6107784431137725, 'f1': 0.4857142857142857, 'number': 167}","{'precision': 0.8490048043925875, 'recall': 0.8970268310369833, 'f1': 0.8723554301833568, 'number': 1379}","{'precision': 0.9413214990138067, 'recall': 0.9573721163490472, 'f1': 0.9492789656887122, 'number': 1994}",0.870319,0.915741,0.892452,0.986914
6,0.009800,0.075848,"{'precision': 0.8562737642585552, 'recall': 0.9147034930950447, 'f1': 0.8845247446975648, 'number': 1231}","{'precision': 0.44387755102040816, 'recall': 0.5209580838323353, 'f1': 0.47933884297520657, 'number': 167}","{'precision': 0.860637509266123, 'recall': 0.8419144307469181, 'f1': 0.8511730205278593, 'number': 1379}","{'precision': 0.9419642857142857, 'recall': 0.9523570712136409, 'f1': 0.9471321695760598, 'number': 1994}",0.876333,0.895619,0.885871,0.986276
7,0.009800,0.076029,"{'precision': 0.8813291139240507, 'recall': 0.9049553208773355, 'f1': 0.8929859719438878, 'number': 1231}","{'precision': 0.35172413793103446, 'recall': 0.6107784431137725, 'f1': 0.4463894967177243, 'number': 167}","{'precision': 0.8559139784946237, 'recall': 0.8658448150833937, 'f1': 0.8608507570295603, 'number': 1379}","{'precision': 0.9468455042225534, 'recall': 0.9558676028084253, 'f1': 0.9513351634639382, 'number': 1994}",0.869811,0.904632,0.886880,0.986308
8,0.003300,0.081212,"{'precision': 0.8884462151394422, 'recall': 0.9057676685621446, 'f1': 0.8970233306516492, 'number': 1231}","{'precision': 0.4152542372881356, 'recall': 0.5868263473053892, 'f1': 0.48635235732009924, 'number': 167}","{'precision': 0.8624185372918175, 'recall': 0.8636693255982596, 'f1': 0.8630434782608696, 'number': 1379}","{'precision': 0.9373482272948033, 'recall': 0.9679037111334002, 'f1': 0.9523809523809524, 'number': 1994}",0.8

The following columns in the evaluation set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: segmented_tokens, segmented_labels. If segmented_tokens, segmented_labels are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4211
  Batch size = 16
Trainer is attempting to log a value of "{'precision': 0.8329604772557793, 'recall': 0.9073923639317628, 'f1': 0.8685847589424572, 'number': 1231}" of type <class 'dict'> for key "eval/LOCATION" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.32388663967611336, 'recall': 0.47904191616766467, 'f1': 0.3864734299516908, 'number': 167}" of type <class 'dict'> for key "eval/MISCELLANEOUS" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribut

Trainer is attempting to log a value of "{'precision': 0.876984126984127, 'recall': 0.8976441917140536, 'f1': 0.8871938980329186, 'number': 1231}" of type <class 'dict'> for key "eval/LOCATION" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.34551495016611294, 'recall': 0.6227544910179641, 'f1': 0.4444444444444444, 'number': 167}" of type <class 'dict'> for key "eval/MISCELLANEOUS" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8268575323790047, 'recall': 0.8796229151559101, 'f1': 0.8524244553759662, 'number': 1379}" of type <class 'dict'> for key "eval/ORGANIZATION" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9341404358353511, 'reca